In [ ]:
# isanlp >= 0.0.7
! pip install -U git+https://github.com/IINemo/isanlp.git

In [1]:
from isanlp import PipelineCommon
from isanlp.processor_razdel import ProcessorRazdel
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd
import razdel

# put the address here ->
address_syntax = ('', 3134)
address_rst = ('', 3335)

# Highly recommended to pre-tokenize texts
def tokenize(text):
    """ Tokenize text, but keep paragraph boundaries. """

    while '\n\n' in text:
        text = text.replace('\n\n', '\n')
    result = []
    for paragraph in text.split('\n'):
        result.append(' '.join([tok.text for tok in razdel.tokenize(paragraph)]))
    return '\n'.join(result).strip()

ppl = PipelineCommon([
    (ProcessorRazdel(), ['text'],
     {'tokens': 'tokens',
      'sentences': 'sentences'}),
    (ProcessorRemote(address_syntax[0], address_syntax[1], '0'),
     ['tokens', 'sentences'],
     {'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
     ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'}),
    (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
     ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {'rst': 'rst'})
])

In [2]:
text = tokenize("Парацетамол (известный в Соединённых Штатах как ацетаминофен) – одно из самых распространённых "
        "лекарств в мире. Из-за своей безопасности он является предпочтительным препаратом для снятия жара "
        "и болевых синдромов. Однако 10 лет назад была высказана гипотеза о том, что использование "
        "парацетамола может увеличить риск развития астмы. Высказывалось предположение, что переход в США "
        "в 80-х гг. с применения аспирина на парацетамол среди детей мог стать причиной увеличения числа детей, "
        "заболевших астмой в течение данного периода.")

In [3]:
%%time

result = ppl(text)

CPU times: user 26 ms, sys: 66 µs, total: 26.1 ms
Wall time: 3.28 s


In [4]:
result['rst']

In [6]:
print(result['rst'][0])

id: 22
text: Парацетамол ( известный в Соединённых Штатах как ацетаминофен ) – одно из самых распространённых лекарств в мире . Из-за своей безопасности он является предпочтительным препаратом для снятия жара и болевых синдромов . Однако 10 лет назад была высказана гипотеза о том , что использование парацетамола может увеличить риск развития астмы . Высказывалось предположение , что переход в США в 80-х гг . с применения аспирина на парацетамол среди детей мог стать причиной увеличения числа детей , заболевших астмой в течение данного периода .
proba: 1.0
relation: contrast
nuclearity: NN
left: Парацетамол ( известный в Соединённых Штатах как ацетаминофен ) – одно из самых распространённых лекарств в мире . Из-за своей безопасности он является предпочтительным препаратом для снятия жара и болевых синдромов .
right: Однако 10 лет назад была высказана гипотеза о том , что использование парацетамола может увеличить риск развития астмы . Высказывалось предположение , что переход в США в 80

In [7]:
def extr_pairs(tree, text):
    pp = []
    if tree.left:
        pp.append([text[tree.left.start:tree.left.end], text[tree.right.start:tree.right.end], tree.relation, tree.nuclearity])
        pp += extr_pairs(tree.left, text)
        pp += extr_pairs(tree.right, text)
    return pp

In [8]:
extr_pairs(result['rst'][0], result['text'])

[['Парацетамол ( известный в Соединённых Штатах как ацетаминофен ) – одно из самых распространённых лекарств в мире . Из-за своей безопасности он является предпочтительным препаратом для снятия жара и болевых синдромов .',
  'Однако 10 лет назад была высказана гипотеза о том , что использование парацетамола может увеличить риск развития астмы . Высказывалось предположение , что переход в США в 80-х гг . с применения аспирина на парацетамол среди детей мог стать причиной увеличения числа детей , заболевших астмой в течение данного периода .',
  'contrast',
  'NN'],
 ['Парацетамол ( известный в Соединённых Штатах как ацетаминофен ) – одно из самых распространённых лекарств в мире .',
  'Из-за своей безопасности он является предпочтительным препаратом для снятия жара и болевых синдромов .',
  'elaboration',
  'NS'],
 ['Из-за своей безопасности',
  'он является предпочтительным препаратом для снятия жара и болевых синдромов .',
  'cause-effect',
  'SN'],
 ['он является предпочтительным пре

In [9]:
from isanlp.annotation_rst import ForestExporter

exporter = ForestExporter(encoding='utf8')
exporter(result['rst'], 'example.rs3')

Output:
    
<div>
<img src="example.rs3.png" width="700"/>
</div>